In [ ]:
!pip install --quiet /kaggle/input/kerasapplications
!pip install --quiet /kaggle/input/efficientnet-git

In [ ]:
#!pip install git+https://github.com/qubvel/efficientnet
# from keras.applications.nasnet import NASNetMobile

In [ ]:
#!pip install ../input/efficientnet-keras-source-code/repository/qubvel-efficientnet-c993591

In [ ]:
!pip install ../input/classification-models-by-qubvel

In [ ]:
import math, os, re, warnings, random, glob
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras.layers as L
import tensorflow.keras.backend as K
from tensorflow.keras import Sequential
import efficientnet.tfkeras as efn

def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'

seed = 0
seed_everything(seed)
warnings.filterwarnings('ignore')

In [ ]:
# TPU or GPU detection
# Detect hardware, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print(f'Running on TPU {tpu.master()}')
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()

AUTO = tf.data.experimental.AUTOTUNE
REPLICAS = strategy.num_replicas_in_sync
print(f'REPLICAS: {REPLICAS}')

In [ ]:
BATCH_SIZE = 32 * REPLICAS
HEIGHT = 512
WIDTH = 512 
CHANNELS = 3
N_CLASSES = 5
TTA_STEPS = 3 # Do TTA if > 0 
MODEL_NAME = 'EfficientNet4'
focal_loss = False
SWA = False
label_smoothing = False

In [ ]:
def data_augment(image, label):
    p_rotation = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_spatial = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_rotate = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_pixel_1 = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_pixel_2 = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_pixel_3 = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_shear = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_crop = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    
    # Shear
    if p_shear > .2:
        if p_shear > .6:
            image = transform_shear(image, HEIGHT, shear=20.)
        else:
            image = transform_shear(image, HEIGHT, shear=-20.)
    # Rotation
    if p_rotation > .2:
        if p_rotation > .6:
            image = transform_rotation(image, HEIGHT, rotation=45.)
        else:
            image = transform_rotation(image, HEIGHT, rotation=-45.)
    # Flips
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    if p_spatial > .75:
        image = tf.image.transpose(image)
    # Rotates
    if p_rotate > .75:
        image = tf.image.rot90(image, k=3) # rotate 270º
    elif p_rotate > .5:
        image = tf.image.rot90(image, k=2) # rotate 180º
    elif p_rotate > .25:
        image = tf.image.rot90(image, k=1) # rotate 90º
    # Pixel-level transforms
    if p_pixel_1 >= .4:
        image = tf.image.random_saturation(image, lower=.7, upper=1.3)
    if p_pixel_2 >= .4:
        image = tf.image.random_contrast(image, lower=.8, upper=1.2)
    if p_pixel_3 >= .4:
        image = tf.image.random_brightness(image, max_delta=.1)
    # Crops
    if p_crop > .7:
        if p_crop > .9:
            image = tf.image.central_crop(image, central_fraction=.6)
        elif p_crop > .8:
            image = tf.image.central_crop(image, central_fraction=.7)
        else:
            image = tf.image.central_crop(image, central_fraction=.8)
    elif p_crop > .4:
        crop_size = tf.random.uniform([], int(HEIGHT*.6), HEIGHT, dtype=tf.int32)
        image = tf.image.random_crop(image, size=[crop_size, crop_size, CHANNELS])

    return image, label

In [ ]:
# data augmentation @cdeotte kernel: https://www.kaggle.com/cdeotte/rotation-augmentation-gpu-tpu-0-96
def transform_rotation(image, height, rotation):
    # input image - is one image of size [dim,dim,3] not a batch of [b,dim,dim,3]
    # output - image randomly rotated
    DIM = height
    XDIM = DIM%2 #fix for size 331
    
    rotation = rotation * tf.random.uniform([1],dtype='float32')
    # CONVERT DEGREES TO RADIANS
    rotation = math.pi * rotation / 180.
    
    # ROTATION MATRIX
    c1 = tf.math.cos(rotation)
    s1 = tf.math.sin(rotation)
    one = tf.constant([1],dtype='float32')
    zero = tf.constant([0],dtype='float32')
    rotation_matrix = tf.reshape(tf.concat([c1,s1,zero, -s1,c1,zero, zero,zero,one],axis=0),[3,3])

    # LIST DESTINATION PIXEL INDICES
    x = tf.repeat( tf.range(DIM//2,-DIM//2,-1), DIM )
    y = tf.tile( tf.range(-DIM//2,DIM//2),[DIM] )
    z = tf.ones([DIM*DIM],dtype='int32')
    idx = tf.stack( [x,y,z] )
    
    # ROTATE DESTINATION PIXELS ONTO ORIGIN PIXELS
    idx2 = K.dot(rotation_matrix,tf.cast(idx,dtype='float32'))
    idx2 = K.cast(idx2,dtype='int32')
    idx2 = K.clip(idx2,-DIM//2+XDIM+1,DIM//2)
    
    # FIND ORIGIN PIXEL VALUES 
    idx3 = tf.stack( [DIM//2-idx2[0,], DIM//2-1+idx2[1,]] )
    d = tf.gather_nd(image, tf.transpose(idx3))
        
    return tf.reshape(d,[DIM,DIM,3])

def transform_shear(image, height, shear):
    # input image - is one image of size [dim,dim,3] not a batch of [b,dim,dim,3]
    # output - image randomly sheared
    DIM = height
    XDIM = DIM%2 #fix for size 331
    
    shear = shear * tf.random.uniform([1],dtype='float32')
    shear = math.pi * shear / 180.
        
    # SHEAR MATRIX
    one = tf.constant([1],dtype='float32')
    zero = tf.constant([0],dtype='float32')
    c2 = tf.math.cos(shear)
    s2 = tf.math.sin(shear)
    shear_matrix = tf.reshape(tf.concat([one,s2,zero, zero,c2,zero, zero,zero,one],axis=0),[3,3])    

    # LIST DESTINATION PIXEL INDICES
    x = tf.repeat( tf.range(DIM//2,-DIM//2,-1), DIM )
    y = tf.tile( tf.range(-DIM//2,DIM//2),[DIM] )
    z = tf.ones([DIM*DIM],dtype='int32')
    idx = tf.stack( [x,y,z] )
    
    # ROTATE DESTINATION PIXELS ONTO ORIGIN PIXELS
    idx2 = K.dot(shear_matrix,tf.cast(idx,dtype='float32'))
    idx2 = K.cast(idx2,dtype='int32')
    idx2 = K.clip(idx2,-DIM//2+XDIM+1,DIM//2)
    
    # FIND ORIGIN PIXEL VALUES 
    idx3 = tf.stack( [DIM//2-idx2[0,], DIM//2-1+idx2[1,]] )
    d = tf.gather_nd(image, tf.transpose(idx3))
        
    return tf.reshape(d,[DIM,DIM,3])

In [ ]:
# Datasets utility functions
def get_name(file_path):
    parts = tf.strings.split(file_path, os.path.sep)
    name = parts[-1]
    return name

def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    return image

def resize_image(image, label):
#     image = tf.image.random_crop(image, size=[HEIGHT, WIDTH, CHANNELS])
    image = tf.image.resize(image, [HEIGHT, WIDTH])
    image = tf.reshape(image, [HEIGHT, WIDTH, CHANNELS])
    return image, label

def process_path(file_path):
    name = get_name(file_path)
    img = tf.io.read_file(file_path)
    img = decode_image(img)
    return img, name

def get_dataset(files_path, shuffled=False, tta=False, extension='jpg'):
    dataset = tf.data.Dataset.list_files(f'{files_path}*{extension}', shuffle=shuffled)
    dataset = dataset.map(process_path, num_parallel_calls=AUTO)
    if tta:
        dataset = dataset.map(data_augment, num_parallel_calls=AUTO)
    dataset = dataset.map(resize_image, num_parallel_calls=AUTO)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO)
    return dataset

def count_data_items(filenames):
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)

In [ ]:
database_base_path = '/kaggle/input/cassava-leaf-disease-classification/'
submission = pd.read_csv(f'{database_base_path}sample_submission.csv')
display(submission.head())

TEST_FILENAMES = tf.io.gfile.glob(f'{database_base_path}test_tfrecords/ld_test*.tfrec')
NUM_TEST_IMAGES = count_data_items(TEST_FILENAMES)
print(f'GCS: test: {NUM_TEST_IMAGES}')

In [ ]:
model_path_list = glob.glob('../input/b4es512is45epo-1e4lr03cutmix512fc03dropextdata/model*.h5') #'/kaggle/input/cassava-leaf-disease-tpu-tensorflow-training/*.h5'
model_path_list.sort()

print('Models to predict:')
print(*model_path_list, sep='\n')

In [ ]:
from tensorflow.keras.applications import DenseNet121, DenseNet201 , DenseNet169
from tensorflow.keras.applications import vgg16
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications import ResNet101 , ResNet101V2 , ResNet152 , ResNet152V2 , ResNet50V2 , ResNet50
from tensorflow.keras.applications import MobileNet , MobileNetV2
from tensorflow.keras.applications import InceptionResNetV2
import tensorflow.keras.layers as L
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.losses import CategoricalCrossentropy
from classification_models.keras import Classifiers


def get_model_generalized(name,input_shape,N_CLASSES):
    
    if name == 'EfficientNet0' :
        wg = '../input/efficientnet-keras-noisystudent-weights-b0b7/efficientnet-b0_noisy-student_notop.h5'
        base_model = efn.EfficientNetB0(weights=wg,
                                        include_top = False,
                                        #pooling='avg',
                                        input_shape=input_shape
                                       )
    elif name == 'EfficientNet1' :
        wg = '../input/efficientnet-keras-noisystudent-weights-b0b7/efficientnet-b1_noisy-student_notop.h5'
        base_model = efn.EfficientNetB1(weights=wg,
                                        include_top = False,
                                        #pooling='avg',
                                        input_shape=input_shape)
    elif name == 'EfficientNet2' :
        wg = '../input/efficientnet-keras-noisystudent-weights-b0b7/efficientnet-b2_noisy-student_notop.h5'
        base_model = efn.EfficientNetB2(weights=wg,
                                        include_top = False,
                                        #pooling='avg',
                                        input_shape=input_shape
                                       )
    elif name == 'EfficientNet4' :
        wg = '../input/efficientnet-keras-noisystudent-weights-b0b7/efficientnet-b4_noisy-student_notop.h5'
        base_model = efn.EfficientNetB4(weights=wg,
                                        include_top = False,
                                        #pooling='avg',
                                        input_shape=input_shape
                                       )
    elif name == 'EfficientNet6' :
        wg = '../input/efficientnet-keras-noisystudent-weights-b0b7/efficientnet-b6_noisy-student_notop.h5'
        base_model = efn.EfficientNetB6(weights=wg,
                                        include_top = False,
                                        #pooling='avg',
                                        input_shape=input_shape
                                       )
    elif name == 'EfficientNet5' :
        wg = '../input/efficientnet-keras-noisystudent-weights-b0b7/efficientnet-b5_noisy-student_notop.h5'
        base_model = efn.EfficientNetB5(weights=wg,
                                        include_top = False,
                                        #pooling='avg',
                                        input_shape=input_shape
                                       )

    elif name == 'EfficientNet7' :
        wg = '../input/efficientnet-keras-noisystudent-weights-b0b7/efficientnet-b7_noisy-student_notop.h5'
        base_model = efn.EfficientNetB7(weights=wg,
                                        include_top = False,
                                        #pooling='avg',
                                        input_shape=input_shape
                                       )
        #base_model.trainable = True
        #for layer in base_model.layers[:-20] :
        #    layer.trainable = True
    elif name == 'EfficientNet3' :
        wg = '../input/efficientnet-keras-noisystudent-weights-b0b7/efficientnet-b3_noisy-student_notop.h5'
        base_model = efn.EfficientNetB3(weights=wg,
                                        include_top = False,
                                        #pooling='avg',
                                        input_shape=input_shape
                                       )
        #base_model.trainable = True
        #for layer in base_model.layers[:-25] :
        #    layer.trainable = True        
            
    elif name == 'DenseNet201' :
        wg = '../input/keras-pretrained-imagenet-weights/densenet201_imagenet_1000_no_top.h5'
        base_model = DenseNet201(weights=wg,include_top=False,input_shape=input_shape)
    elif name == 'DenseNet169' :
        wg = '../input/keras-pretrained-imagenet-weights/densenet169_imagenet_1000_no_top.h5'
        base_model = DenseNet169(weights=wg,include_top=False,input_shape=input_shape)
        #base_model.trainable = True
    elif name == 'DenseNet121' :
        wg = '../input/keras-pretrained-imagenet-weights/densenet121_imagenet_1000_no_top.h5'
        base_model = DenseNet121(weights=wg,include_top=False,input_shape=input_shape)
        #for layer in base_model.layers[:-25] :
        #    layer.trainable = True
    elif name == 'MobileNet' :
        wg = '../input/keras-pretrained-imagenet-weights/mobilenet_imagenet_1000_no_top.h5'
        base_model = MobileNet(weights = wg, include_top=False,input_shape=input_shape)
    elif name == 'Inception' :
        wg = '../input/keras-pretrained-imagenet-weights/inceptionv3_imagenet_1000_no_top.h5'
        base_model = InceptionV3(weights =wg,include_top=False,input_shape=input_shape)
    elif name == 'ResNet50' :
        wg = '../input/keras-pretrained-imagenet-weights/resnet50_imagenet_1000_no_top.h5'
        base_model = ResNet50(weights = wg,include_top=False,input_shape=input_shape)
    elif name == 'ResNet50V2' :
        base_model = ResNet50V2(weights = wg,include_top=False,input_shape=input_shape)
    elif name == 'ResNet101' :
        wg = '../input/keras-pretrained-imagenet-weights/resnet101_imagenet_1000_no_top.h5'
        base_model = ResNet101(weights = wg,include_top=False,input_shape=input_shape)
    elif name == 'ResNet101V2' :
        base_model = ResNet101V2(weights =wg,include_top=False,input_shape=input_shape)
    elif name == 'ResNet152' :
        wg = '../input/keras-pretrained-imagenet-weights/resnet152_imagenet_1000_no_top.h5'
        base_model = ResNet152(weights = wg,include_top=False,input_shape=input_shape)
    elif name == 'ResNet152V2' :
        base_model = ResNet152V2(weights = wg,include_top=False,input_shape=input_shape)
    elif name == 'Incepresnet' :
        wg = '../input/keras-pretrained-imagenet-weights/inceptionresnetv2_imagenet_1000_no_top.h5'
        base_model = InceptionResNetV2(weights =wg,include_top=False,input_shape=input_shape) 
        #base_model.trainable = True
        #for layer in base_model.layers[:-25] :
        #    layer.trainable = True
    elif name == 'seresnet18' : 
        model, preprocess_input = Classifiers.get('seresnet18')
        wg = '../input/keras-pretrained-imagenet-weights/seresnet18_imagenet_1000_no_top.h5'
        base_model = model(input_shape=input_shape, weights=wg, include_top=False)
    elif name == 'seresnet34' : 
        model, preprocess_input = Classifiers.get('seresnet34')
        wg = '../input/keras-pretrained-imagenet-weights/seresnet34_imagenet_1000_no_top.h5'
        base_model = model(input_shape=input_shape, weights=wg, include_top=False)
    elif name == 'seresnet50' : 
        model, preprocess_input = Classifiers.get('seresnet50')
        wg = '../input/keras-pretrained-imagenet-weights/seresnet50_imagenet_1000_no_top.h5'
        base_model = model(input_shape=input_shape, weights=wg, include_top=False)
    elif name == 'seresnet101' : 
        model, preprocess_input = Classifiers.get('seresnet101')
        wg = '../input/keras-pretrained-imagenet-weights/seresnet101_imagenet_1000_no_top.h5'
        base_model = model(input_shape=input_shape, weights=wg, include_top=False)
    elif name == 'seresnet152' : 
        model, preprocess_input = Classifiers.get('seresnet152')
        wg = '../input/keras-pretrained-imagenet-weights/seresnet152_imagenet_1000_no_top.h5'
        base_model = model(input_shape=input_shape, weights=wg, include_top=False)
    elif name == 'seresnext50' : 
        model, preprocess_input = Classifiers.get('seresnext50')
        wg = '../input/keras-pretrained-imagenet-weights/seresnext50_imagenet_1000_no_top.h5'
        base_model = model(input_shape=input_shape, weights=wg, include_top=False)
    elif name == 'seresnext101' : 
        model, preprocess_input = Classifiers.get('seresnext101')
        wg = '../input/keras-pretrained-imagenet-weights/seresnext101_imagenet_1000_no_top.h5'
        base_model = model(input_shape=input_shape, weights=wg, include_top=False)
    x = base_model.output
    x = L.GlobalAveragePooling2D()(x)
    x = L.Dense(512,activation='relu')(x)
    x = L.Dropout(0.3)(x)
    x = L.Dense(256,activation='relu')(x)
    x = L.Dropout(0.2)(x)
    predictions = L.Dense(N_CLASSES,activation='softmax')(x)
    model = Model(inputs = base_model.input, outputs=predictions) 
    
    '''if focal_loss : 
        loss= tfa.losses.SigmoidFocalCrossEntropy(reduction=tf.keras.losses.Reduction.AUTO)
    elif label_smoothing :
        loss=CategoricalCrossentropy(label_smoothing=label_smoothing)
    else :
        loss = 'categorical_cross_entropy'
    if SWA :
        opt = tf.keras.optimizers.Adam(lr=1e-5) # roll back
        opt = tfa.optimizers.SWA(opt)
    else :
        opt = 'adam'
        
    optimizer = tf.keras.optimizers.Adam(lr=LEARNING_RATE)
    model.compile(optimizer=optimizer, 
                  loss=losses.SparseCategoricalCrossentropy(), 
                  metrics=['sparse_categorical_accuracy'])'''
    return model

with strategy.scope():
    model = get_model_generalized(MODEL_NAME,(None, None, CHANNELS), N_CLASSES)

In [ ]:
files_path = f'{database_base_path}/test_images/'
test_preds = np.zeros((len(os.listdir(files_path)), N_CLASSES))


for model_path in model_path_list:
    print(model_path)
    K.clear_session()
    model.load_weights(model_path)

    if TTA_STEPS > 0:
        test_ds = get_dataset(files_path, tta=True)
        for step in range(TTA_STEPS):
            print(f'TTA step {step+1}/{TTA_STEPS}')
            x_test = test_ds.map(lambda image, image_name: image)
            test_preds += model.predict(x_test) / (TTA_STEPS * len(model_path_list))
    else:
        test_ds = get_dataset(files_path, tta=False)
        x_test = test_ds.map(lambda image, image_name: image)
        test_preds += model.predict(x_test) / len(model_path_list)
    
test_preds = np.argmax(test_preds, axis=-1)
image_names = [img_name.numpy().decode('utf-8') for img, img_name in iter(test_ds.unbatch())]

In [ ]:
submission = pd.DataFrame({'image_id': image_names, 'label': test_preds})
submission.to_csv('submission.csv', index=False)
display(submission.head())

In [ ]:
test_preds